##### Reinforcement Learning &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Homework #3

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Sarsa

## Description

For this assignment,  you will build a Sarsa agent which will learn policies in the [OpenAI Gym](http://gym.openai.com/docs/) Frozen Lake environment.  [OpenAI Gym](http://gym.openai.com/docs/) is a platform where users can test their RL algorithms on a selection of carefully crafted environments.  As we will continue to use [OpenAI Gym](http://gym.openai.com/docs/) through Project 2, this assignment also provides an opportunity to familiarize yourself with its interface.

Frozen Lake is a grid world environment that is highly stochastic,  where the agent must cross a slippery frozen  lake  which  has  deadly  holes  to  fall  through.   The  agent  begins  in  the  starting  state `S` and  is  given  a reward of `1` if it reaches the goal state `G`.  A reward of `0` is given for all other transitions.

The agent can take one of four possible moves at each state (left, down, right, or up).  The frozen cells `F` are slippery, so the agent’s actions succeed only `1/3` of the time, while the other `2/3` are split evenly between the two directions orthogonal to the intended direction.  If the agent lands in a hole `H`, then the episode terminates. You will be given a randomized Frozen Lake map with a corresponding set of parameters to train your Sarsa agent with.


$$\text{Sarsa } (S_t, A_t, R_{t+1}, S_{t+1}, A_{t+1})$$

Sarsa uses temporal-difference learning to form a model-free on-policy reinforcement-learning algorithm that solves the *control* problem. It is model free because it does not need and does not use a model of the environment, namely neither a transition nor reward function; instead, Sarsa samples transitions and rewards online.

It is on-policy because it learns about the same policy that generates its behaviors (this is in contrast to *Q-learning*).  That is, Sarsa estimates the action-value function of its behavior policy.  In this homework,  you will not be training a Sarsa agent to approximate the *optimal* action-value function; instead, the hyperparameters of both the exploration strategy and the algorithm will be given to you as input — the goal being to verify that your SARSA agent is correctly implemented.

## Procedure

Attention to detail to each of the following points is required:

- You must use Python and the library NumPy for this homework *python 3.6.x* and *numpy==1.18.0*, or more recent version.

- Install OpenAI Gym (e.g.pip install gym) *gym==0.17.2*

- The Frozen Lake environment has been instantiated for you.
- The pertinent random number generators have been seeded for you. Do *not* use the Python standard library’s *random* library.

- Implement your Sarsa agent using an $\epsilon$-greedy behavioral policy.  Specifically, you must use *numpy.random.random* to  choose  whether  or  not  the  action  is  greedy,  and *numpy.random.randint* to select the random action.

- Initialize the agent’s Q-table to zeros.

- Train your agent using the given input parameters.  The input *amap* is the Frozen Lake map that you need to resize and provide to the *desc* attribute when you instantiate your environment.  The input *gamma* is the discount rate.  The input *alpha* is the learning rate.  The input *epsilon* is the parameter for the $\epsilon$-greedy behavior strategy your Sarsa agent will use.  Specifically, an action should be selected uniformly at random if a random number drawn uniformly between 0 and 1 is less than $\epsilon$.  If the greedy action is selected,  the  action  with  lowest  index  should  be  selected  in  case  of  ties.   The  input `n_episodes` is  the number of episodes to train your agent.  Finally, *seed* is the number used to seed both Gym’s random number generator and NumPy’s random number generator.

- Your Sarsa implementation should select the action corresponding to the next state the  agent  will  visit *even when* that  next  state  is  a  terminal  state.

- You should return the greedy policy with respect to the Q-function obtained by your Sarsa agent after the completion of the final episode.  Specifically, the policy should be expressed as a string ofcharacters: **<, v, >, ^,** representing left, down, right, and up, respectively.  The ordering of the actions in the output should reflect the ordering of states in *amap*. 

## Resources

The concepts explored in this homework are covered by:

-   Lesson 4: Convergence

-   Chapter 6 (6.4 Sarsa:  On-policy TD Control) of
    http://incompleteideas.net/book/the-book-2nd.html


## Submission

-   The due date is indicated on the Syllabus page for this assignment.
-   	Use the template code to implement your wor.
-   	Please use python 3.6.x or more recent version, gym==0.17.2 and numpy==1.18.0 or more recent version, and you can use any core library  (i.e., anything in the Python standard library). No other library can be used. 


In [27]:
#################
# DO NOT REMOVE
# Versions
# numpy==1.18.0
# gym==0.17.2
# 刘兴琰
# 202264690069
################

import gym
import numpy as np
from math import sqrt
from gym.envs import toy_text

class FrozenLakeAgent(object):
    def __init__(self):
        pass

    def amap_to_gym(self, amap='FFGG'):
        """
        Converts the input map (amap) string into a Gym environment.
        
        Parameters:
        - amap: A string representing the Frozen Lake map (e.g., 'SFFFHFFF...').
               'S' - Start state, 'F' - Frozen (safe), 'H' - Hole (fall), 'G' - Goal.
               
        Returns:
        - A Frozen Lake environment instantiated with the custom map provided by amap.
        """
        amap = np.asarray(amap, dtype='c')   
        side = int(sqrt(amap.shape[0]))   
        amap = amap.reshape((side, side))   
        return gym.make('FrozenLake-v0', desc=amap).unwrapped   
    def epsilon_greedy(self, Q, state, epsilon, n_actions):
        """
        Selects an action using the epsilon-greedy policy.
        
        Parameters:
        - Q: The Q-table (state-action value table).
        - state: The current state of the agent.
        - epsilon: Probability of selecting a random action for exploration (controls exploration vs exploitation).
        - n_actions: Total number of actions available in the environment.
        
        Returns:
        - The action chosen based on the epsilon-greedy strategy. It chooses a random action with probability epsilon,
          otherwise it selects the action with the highest Q-value for the current state (exploitation).
        """
        if np.random.random() < epsilon:
            return np.random.randint(n_actions)   
        else:
            return np.argmax(Q[state])   

    def solve(self, amap, gamma, alpha, epsilon, n_episodes, seed):
        """
        Train the SARSA agent using the provided parameters and return the learned greedy policy.
        
        Parameters:
        - amap: The map (string) for Frozen Lake (as explained above).
        - gamma: Discount factor (determines the weight of future rewards, closer to 1 values future rewards higher).
        - alpha: Learning rate (determines the extent to which newly acquired information overrides old information).
        - epsilon: Epsilon for epsilon-greedy policy (higher value means more exploration).
        - n_episodes: Number of episodes to run for training (each episode is one full run of the environment).
        - seed: Random seed for reproducibility of the experiment.
        
        Returns:
        - A policy string of characters representing the greedy action for each state ('<', 'v', '>', '^').
          The actions map to left, down, right, and up, respectively.
        """
        env = self.amap_to_gym(amap)
        np.random.seed(seed)  
        env.seed(seed)  
        
        n_actions = env.action_space.n  
        n_states = env.observation_space.n  

        Q = np.zeros((n_states, n_actions))

 
        for episode in range(n_episodes):
            state = env.reset()   
            action = self.epsilon_greedy(Q, state, epsilon, n_actions)   
            
            done = False   
            while not done:
                next_state, reward, done, _ = env.step(action)   
                next_action = self.epsilon_greedy(Q, next_state, epsilon, n_actions)   
                
  
                Q[state, action] += alpha * (reward + gamma * Q[next_state, next_action] - Q[state, action])

 
                state, action = next_state, next_action
        
 
        policy = ''
        actions_map = ['<', 'v', '>', '^']   
        for state in range(n_states):
            best_action = np.argmax(Q[state])   
            policy += actions_map[best_action]   
        
        return policy

## Test cases

In [28]:

## DO NOT MODIFY THIS CODE.

import unittest


class TestQNotebook(unittest.TestCase):
    def setUp(self):
        self.agent = FrozenLakeAgent()

    def test_case_1(self):
        example1 = self.agent.solve(
            amap='SFFFHFFFFFFFFFFG',
            gamma=1.0,
            alpha=0.25,
            epsilon=0.29,
            n_episodes=14697,
            seed=741684
        )
        assert(example1 == '^vv><>>vvv>v>>><')

    def test_case_2(self):
        example2 = self.agent.solve(
            amap='SFFFFHFFFFFFFFFFFFFFFFFFG',
            gamma=0.91,
            alpha=0.12,
            epsilon=0.13,
            n_episodes=42271,
            seed=983459
        )
        assert(example2 == '^>>>><>>>vvv>>vv>>>>v>>^<')

    def test_case_3(self):
        example3 = self.agent.solve(
            amap='SFFG',
            gamma=1.0,
            alpha=0.24,
            epsilon=0.09,
            n_episodes=49553,
            seed=20240
        )
        assert(example3 == '<<v<')

    def test_case_4(self):
        example4 = self.agent.solve(
            amap='SFFHHFFHHFFHHFFG',
            gamma=0.99,
            alpha=0.5,
            epsilon=0.29,
            n_episodes=23111,
            seed=44323
        )
        assert(example4=='^><<<>^<<><<<>^<')

    def test_case_5(self):
        example5 = self.agent.solve(
            amap='SFFFFHFFFHHFFFFFFFFHHFFFG',
            gamma=0.88,
            alpha=0.15,
            epsilon=0.16,
            n_episodes=112312,
            seed=6854343
        )
        assert(example5 == '^>><^<>><<<>v<^v>v<<<>vv<')

unittest.main(argv=[''], verbosity=2, exit=False) 

test_case_1 (__main__.TestQNotebook.test_case_1) ... ok
test_case_2 (__main__.TestQNotebook.test_case_2) ... ok
test_case_3 (__main__.TestQNotebook.test_case_3) ... ok
test_case_4 (__main__.TestQNotebook.test_case_4) ... ok
test_case_5 (__main__.TestQNotebook.test_case_5) ... ok

----------------------------------------------------------------------
Ran 5 tests in 26.279s

OK
